In [1]:
import csv
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from data import BodyPart 
import tensorflow as tf


In [2]:
# loading final csv file
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    df.drop(['filename'],axis=1, inplace=True)
    classes = df.pop('class_name').unique()
    y = df.pop('class_no')
    
    X = df.astype('float64')
    y = keras.utils.to_categorical(y)
    
    return X, y, classes


def get_center_point(landmarks, left_bodypart, right_bodypart):
    """Calculates the center point of the two given landmarks."""
    left = tf.gather(landmarks, left_bodypart.value, axis=1)
    right = tf.gather(landmarks, right_bodypart.value, axis=1)
    center = left * 0.5 + right * 0.5
    return center


def get_pose_size(landmarks, torso_size_multiplier=2.5):
    """Calculates pose size.
    It is the maximum of two values:
    * Torso size multiplied by `torso_size_multiplier`
    * Maximum distance from pose center to any pose landmark
    """
    # Hips center
    hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    # Shoulders center
    shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                      BodyPart.RIGHT_SHOULDER)

    # Torso size as the minimum body size
    torso_size = tf.linalg.norm(shoulders_center - hips_center)
    # Pose center
    pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                     BodyPart.RIGHT_HIP)
    pose_center_new = tf.expand_dims(pose_center_new, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to
    # perform substraction
    pose_center_new = tf.broadcast_to(pose_center_new,
                                    [tf.size(landmarks) // (17*2), 17, 2])

    # Dist to pose center
    d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                name="dist_to_pose_center")
    # Max dist to pose center
    max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

    # Normalize scale
    pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)
    return pose_size



def normalize_pose_landmarks(landmarks):
    """Normalizes the landmarks translation by moving the pose center to (0,0) and
    scaling it to a constant pose size.
  """
  # Move landmarks so that the pose center becomes (0,0)
    pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    pose_center = tf.expand_dims(pose_center, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to perform
    # substraction
    pose_center = tf.broadcast_to(pose_center, 
                                [tf.size(landmarks) // (17*2), 17, 2])
    landmarks = landmarks - pose_center

    # Scale the landmarks to a constant pose size
    pose_size = get_pose_size(landmarks)
    landmarks /= pose_size
    return landmarks


def landmarks_to_embedding(landmarks_and_scores):
    """Converts the input landmarks into a pose embedding."""
    # Reshape the flat input into a matrix with shape=(17, 3)
    reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

    # Normalize landmarks 2D
    landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])
    # Flatten the normalized landmark coordinates into a vector
    embedding = keras.layers.Flatten()(landmarks)
    return embedding


def preprocess_data(X_train):
    processed_X_train = []
    for i in range(X_train.shape[0]):
        embedding = landmarks_to_embedding(tf.reshape(tf.convert_to_tensor(X_train.iloc[i]), (1, 51)))
        processed_X_train.append(tf.reshape(embedding, (34)))
    return tf.convert_to_tensor(processed_X_train)


In [3]:
X, y, class_names = load_csv('train_data.csv')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
X_test, y_test, _ = load_csv('test_data.csv')

In [6]:
X

,NOSE_x,NOSE_y,NOSE_score,LEFT_EYE_x,LEFT_EYE_y,LEFT_EYE_score,RIGHT_EYE_x,RIGHT_EYE_y,RIGHT_EYE_score,LEFT_EAR_x,...,LEFT_KNEE_score,RIGHT_KNEE_x,RIGHT_KNEE_y,RIGHT_KNEE_score,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_score,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_score
0,158.0,91.0,0.486216,159.0,86.0,0.567007,154.0,86.0,0.471732,155.0,...,0.652461,166.0,223.0,0.659204,152.0,259.0,0.501737,148.0,262.0,0.536225
1,138.0,91.0,0.602957,143.0,87.0,0.507938,139.0,87.0,0.732790,155.0,...,0.859733,125.0,220.0,0.879398,152.0,263.0,0.531063,147.0,259.0,0.663975
2,165.0,100.0,0.674503,165.0,95.0,0.771247,161.0,95.0,0.767396,160.0,...,0.841010,173.0,226.0,0.892319,155.0,262.0,0.701865,152.0,263.0,0.707526
3,134.0,101.0,0.498701,138.0,96.0,0.598086,134.0,95.0,0.545088,151.0,...,0.705889,120.0,224.0,0.837559,150.0,265.0,0.762652,145.0,261.0,0.616944
4,165.0,102.0,0.640302,166.0,99.0,0.595476,162.0,98.0,0.702522,160.0,...,0.780940,173.0,228.0,0.828628,156.0,263.0,0.630904,152.0,264.0,0.672577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,183.0,196.0,0.514371,187.0,190.0,0.481944,178.0,191.0,0.552120,193.0,...,0.423767,141.0,179.0,0.197733,127.0,106.0,0.345544,127.0,106.0,0.509773
1508,120.0,195.0,0.396955,124.0,191.0,0.564490,114.0,191.0,0.470277,134.0,...,0.804588,156.0,130.0,0.362383,172.0,105.0,0.810996,172.0,104.0,0.381398
1509,128.0,186.0,0.231476,132.0,184.0,0.505299,122.0,182.0,0.578729,144.0,...,0.217305,141.0,202.0,0.400085,165.0,194.0,0.184848,142.0,226.0,0.451130
1510,160.0,117.0,0.552652,166.0,111.0,0.740929,155.0,112.0,0.756139,174.0,...,0.249607,163.0,201.0,0.796670,194.0,127.0,0.101683,192.0,126.0,0.113600


In [7]:
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [5]:
class_names

array(['chair', 'cobra', 'dog', 'no_pose', 'shoudler_stand', 'traingle',
       'tree', 'warrior'], dtype=object)

In [4]:
processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)

In [8]:
inputs = tf.keras.Input(shape=(34))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)

model = keras.Model(inputs, outputs)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [44]:
processed_X_train[-1]

<tf.Tensor: shape=(34,), dtype=float32, numpy=
array([ 0.16229115, -0.23299225,  0.16229115, -0.25227436,  0.13979535,
       -0.25227436,  0.13979535, -0.25548804,  0.09480374, -0.25227436,
        0.14300904, -0.20728275,  0.03053002, -0.1976417 ,  0.23620594,
       -0.29405227,  0.08516268, -0.27477017,  0.29405227, -0.3808218 ,
        0.15907747, -0.37760812,  0.04017108, -0.00482053, -0.04017108,
        0.00482053,  0.16550483,  0.1044448 ,  0.11729954,  0.11408585,
        0.08837637,  0.25227436,  0.06266688,  0.2651291 ], dtype=float32)>

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 34)]              0         
                                                                 
 dense (Dense)               (None, 128)               4480      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 13,256
Trainable params: 13,256
Non-trainable p

In [10]:
# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
checkpoint_path = "weights.best.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)

In [11]:
# Start training
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=200,
                    batch_size=16,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])

--------------TRAINING----------------
Epoch 1/200
69/81 [========================>.....] - ETA: 0s - loss: 1.7477 - accuracy: 0.4257
Epoch 00001: val_accuracy improved from -inf to 0.49339, saving model to weights.best.hdf5
81/81 [==============================] - 3s 4ms/step - loss: 1.6978 - accuracy: 0.4366 - val_loss: 1.2321 - val_accuracy: 0.4934
Epoch 2/200
70/81 [========================>.....] - ETA: 0s - loss: 1.1446 - accuracy: 0.5643
Epoch 00002: val_accuracy improved from 0.49339 to 0.68282, saving model to weights.best.hdf5
81/81 [==============================] - 0s 3ms/step - loss: 1.1414 - accuracy: 0.5626 - val_loss: 0.9032 - val_accuracy: 0.6828
Epoch 3/200
69/81 [========================>.....] - ETA: 0s - loss: 0.9399 - accuracy: 0.6685
Epoch 00003: val_accuracy improved from 0.68282 to 0.80617, saving model to weights.best.hdf5
81/81 [==============================] - 0s 3ms/step - loss: 0.9186 - accuracy: 0.6786 - val_loss: 0.7061 - val_accuracy: 0.8062
Epoch 4/20

Epoch 57/200
72/81 [=========================>....] - ETA: 0s - loss: 0.0638 - accuracy: 0.9792
Epoch 00057: val_accuracy did not improve from 0.99559
81/81 [==============================] - 0s 3ms/step - loss: 0.0759 - accuracy: 0.9782 - val_loss: 0.0266 - val_accuracy: 0.9912
Epoch 58/200
72/81 [=========================>....] - ETA: 0s - loss: 0.0627 - accuracy: 0.9835
Epoch 00058: val_accuracy did not improve from 0.99559
81/81 [==============================] - 0s 3ms/step - loss: 0.0660 - accuracy: 0.9829 - val_loss: 0.0294 - val_accuracy: 0.9912
Epoch 59/200
71/81 [=========================>....] - ETA: 0s - loss: 0.0606 - accuracy: 0.9806
Epoch 00059: val_accuracy did not improve from 0.99559
81/81 [==============================] - 0s 3ms/step - loss: 0.0638 - accuracy: 0.9798 - val_loss: 0.0302 - val_accuracy: 0.9912
Epoch 60/200
70/81 [========================>.....] - ETA: 0s - loss: 0.0784 - accuracy: 0.9759
Epoch 00060: val_accuracy did not improve from 0.99559
81/81 [==

In [12]:
print('-----------------EVAUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)



-----------------EVAUATION----------------
28/28 [==============================] - 0s 2ms/step - loss: 0.0082 - accuracy: 0.9954
LOSS:  0.008213489316403866
ACCURACY:  0.9954389929771423


In [13]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model\assets
